In [66]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

nav = webdriver.Chrome()

tabela_produtos = pd.read_excel("./buscas.xlsx")

display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [67]:
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    
    lista_ofertas = []

    # pesquisando pelo produto no google
    nav.get("https://www.google.com/")
    nav.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea').send_keys(produto, Keys.ENTER)

    # entrar na aba shopping
    elemento = nav.find_element("xpath", '/html/body/div[4]/div/div[5]/div/div[1]/div/div/div[1]/div/div/div/div/div[1]/div[2]/a')
    elemento.click()

    # pegando as informações dos produtos
    lista_resultados = nav.find_elements("class name", "i0X6df")

    for resultado in lista_resultados:
        nome = resultado.find_element("class name", "tAxDx").text
        nome = nome.lower()

        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True

        # analisar se ele tem TODOS os termos do nome do produto
        tem_todos_termos_produtos = True
        for palavra in lista_termos_nome_produto:
            if palavra not in nome:
                tem_todos_termos_produtos = False

        # selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produtos = True
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = resultado.find_element("class name", "a8Pemb").text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)

            preco_minimo = float(preco_minimo)
            preco_maximo = float(preco_maximo)

            # se o preco está entre preco_minimo e preco_maximo
            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = resultado.find_element('class name', 'bONr3b')
                elemento_pai = elemento_referencia.find_element('xpath', '..') #pegando o elemento que vem logo antes dele ('pai')
                link = elemento_pai.get_attribute('href')
                lista_ofertas.append((nome, preco, link))
    
    return lista_ofertas

In [68]:
produto = "iphone 12 64 gb"
termos_banidos = "mini watch"
preco_minimo = 3000
preco_maximo = 3500

lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
display(lista_ofertas_google_shopping)

3


[('apple iphone 12 64gb 6.1 azul',
  3150.99,
  'https://www.google.com/url?url=https://www.tradeinn.com/techinn/pt/apple-iphone-12-64gb-6.1/137821872/p%3Futm_source%3Dgoogle_products%26utm_medium%3Dmerchant%26id_producte%3D11865359%26country%3Dbr%26srsltid%3DAfmBOoqvW8l2WQLeCx6UUr-bV_BENnVyw2HBohl9gBeee-VvPp06ArHrbYI&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwjR9t6j2byFAxWHppUCHeCGCxIQgOUECL4Q&usg=AOvVaw324-tuAbXJPfXbSCgfcaVG'),
 ('iphone 12 64gb- branco, , color branco',
  3225.0,
  'https://www.google.com/url?url=https://www.trackerstore.com.br/MLB-3569775671-iphone-12-64gb-_JM%3Fvariation%3D179700168252%26srsltid%3DAfmBOop6E6rHkuJMauKcYDhkqdecbCyDxHxT-65TXxp1aW9SMn6yJFaZQ3c&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwjR9t6j2byFAxWHppUCHeCGCxIQgOUECNMT&usg=AOvVaw210oMp4PRLsxpRB10U3rY_'),
 ('iphone 12 apple 64gb, preto, tela 6.1, câmera dupla 12mp, ios - mgj53br/a',
  3499.0,
  'https://www.google.com/url?url=https://www.somemorias.com.br/iphone-12-apple-64gb-preto-tela-61-camera-